# Занятие 11
# Прикладная алгебра и численные методы
## Итерационные методы решения СЛАУ

In [1]:
import numpy as np
import numpy.linalg
import scipy.linalg

## Метод простой итерации
Приведем систему уравнений
\begin{align*}
A x = b, \quad \det A\neq 0,
\end{align*}
к виду
\begin{align*}
x=P x + b,\quad(1)%\label{sys3}
\end{align*}
положив, например, $P = E - A$.

Выберем произвольный вектор начального приближения $x^0$ и
построим итерационную последовательность по схеме
\begin{align*}
 x^{k + 1} = P x^k + b, \quad k = 1, 2, \ldots\quad(2)%\label{sys_seq}
\end{align*}

Достаточные условия сходимости итерационной последовательности дают
следующие теоремы.


### Теорема  1.
Если $||P||<1$, то система уравнений (1) имеет
единственное решение $ x^*$ и итерационная последовательность
(2) сходится к  $ x^*$, причем
\begin{align*}
|x^k - x^*|\le\frac{|x^{k + 1}- x^k|}{1-||P||}.
\end{align*}
### Следствие 1 из Теоремы 1.
Если $||P||<1$, то 
\begin{align*}
|x^k - x^*|\le\frac{||P||^k}{1-||P||}|x^1 - x^0|.
\end{align*}
### Следствие 2 из Теоремы 1.
Если $||P||<1$, то $|x^k - x^*|\le\varepsilon$, если
\begin{align*}
k\ge\log_{||P||}\left(\varepsilon\frac{1-||P||}{|x^1 - x^0|}\right).
\end{align*}
### Следствие 3 из Теоремы 1.
Если $||P||<1$, и $x^0 = b$, то $|x^k - x^*|\le\varepsilon$, если
\begin{align*}
k\ge\log_{||P||}\left(\varepsilon\frac{1-||P||}{|b|}\right).
\end{align*}
### Теорема 2.
Пусть система уравнений (1) имеет единственное решение
$ x^*$, в этом случае итерационная последовательность
(2) сходится к $ x^*$ при любом начальном приближении
$ x^0$, тогда и только тогда, когда все собственные значения
матрицы $P$ по модулю меньше 1 (т.е. спектральный радиус меньше 1).

Теорема 2 дает более общие условия сходимости метода простой
итерации.

## Пример 1
Решим СЛАУ $AX = b$, где
$$
A = 0.05
\left(
\begin{matrix}
10 & 2 & 0 & 0 & 0\\
3 & 10 & 2 & 0 & 0 \\
0 & 3 & 12 & 2 & 0\\
0 & 0 & 3 & 15 & 2 \\
0 &0 & 0 & 3 & 11
\end{matrix}
\right),
\quad
b = 0.1
\left(
\begin{matrix}
14\\
25\\
20\\
18 \\
3
\end{matrix}
\right), \mbox{ точное решение }
x = \left(
\begin{matrix}
2\\
4\\
2\\
2 \\
0
\end{matrix}
\right).
$$

In [14]:
A1 = 0.05 * np.array([[10, 2, 0, 0, 0],
                      [3, 10, 2, 0, 0],
                      [0, 3, 12, 2, 0],
                      [0, 0, 3, 15, 2],
                      [0, 0, 0, 3, 11]])
b1 = 0.1 * np.array([[14], [25], [20], [18], [3]])
x1 = np.linalg.solve(A1, b1)
print(f'np.linalg.solve: x = {np.round(x1.T, 5)}')
x1_exact = np.array([[2], [4], [2], [2], [0]])
x1_exact = np.array([2, 4, 2, 2, 0])
b1 = A1 @ x1_exact
P1 = np.eye(A1.shape[0]) - A1
print(f"""b = {b1.T},
Точное решение {x1_exact.T}
Норма матрицы P = E - A равна {np.linalg.norm(P1, ord=2).round(2)}""")

np.linalg.solve: x = [[ 2.  4.  2.  2. -0.]]
b = [1.4 2.5 2.  1.8 0.3],
Точное решение [2 4 2 2 0]
Норма матрицы P = E - A равна 0.67


In [21]:
def iteration_simple(A, b, x0, eps=0.001):
    P = np.eye(A.shape[0]) - A
    normP = np.linalg.norm(P, ord=2)
    x_prev = x0
    x_next = P @ x0 + b
    while np.linalg.norm(x_prev - x_next, ord=2) / (1 - normP) >= eps:
        x_prev = x_next
        x_next = P @ x_prev + b
    return  x_next

x_next = iteration_simple(A1, b1, b1, eps=0.01)
print(f"""Решение x = {np.round(x_next, 3).T},
AX - b = {np.round(A1 @ x_next - b1, 3).T}
|xk - x*| = {np.linalg.norm(x_next - x1_exact, ord=2).round(3)}""")    

Решение x = [2.002e+00 3.996e+00 2.003e+00 1.999e+00 1.000e-03],
AX - b = [ 0.001 -0.001  0.001 -0.     0.   ]
|xk - x*| = 0.005


Модифицированный код того же самого, норма другая:

In [22]:
def iteration_simple1(A, b, x0, eps=0.001):
    P = np.eye(A.shape[0]) - A
    eps1 = eps * (1 - np.linalg.norm(P, ord=np.inf))
    x_prev = x0
    x_next = x0 + np.ones(x0.shape[0])
    while np.linalg.norm(x_prev - x_next, ord=np.inf) >= eps1:
        x_prev = x_next  
        x_next = P @ x_prev + b
    return  x_next

x_next = iteration_simple(A1, b1, b1, eps=0.01)
print(f"""Решение x = {np.round(x_next, 3).T},
AX - b = {np.round(A1 @ x_next - b1, 3).T}
|xk - x*| = {np.linalg.norm(x_next - x1_exact, ord=np.inf).round(3)}""")

Решение x = [2.002e+00 3.996e+00 2.003e+00 1.999e+00 1.000e-03],
AX - b = [ 0.001 -0.001  0.001 -0.     0.   ]
|xk - x*| = 0.004


## Итерационные методы решения СЛАУ

Рассматриваем решение СЛАУ
$$
Ax = b, \quad A = L + D + R,
$$
где $L$ - левая (нижняя) треугольная матрица, состоящая из элементов матрицы $A$, находящихся под главной диагональю  $A$, 

$D$ - диагональная матрица, на диагонали - диагональные элементы $A$,

$R$  - правая (верхняя) треугольная матрица, состоящая из элементов $A$, находящихся над главной диагональю матрицы $A$.
## Метод Якоби
$$
D(x^{k + 1} - x^k) + Ax^k = b,\quad x^{k + 1} = (E - D^{-1}A) x^k + \hat b,\quad \hat b = D^{-1}b 
$$
## Метод Гаусса-Зейделя
$$
(D + L)(x^{k + 1} - x^k) + Ax^k = b
$$
## Метод верхней релаксации
$$
(D + \tau L)\frac{x^{k + 1} - x^k}{\tau} + Ax^k = b
$$
### Утверждение
Если $A = A^* > 0$, то метод Зейделя сходится.
### Утверждение
Если все собственные числа матрицы $A$ положительны, то оптимальное значения $\tau$ равно
$$
\tau = \frac{2}{\lambda_{min} + \lambda_{max}}
$$
## Пример 2
Представим матрицу $A$ в виде суммы $A = L + D + R$.

scipy.linalg.tril(matr, k) возвращает нижнюю треугольную матрицу, у которой все элементы над $k$-й диагональю равны 0.

scipy.linalg.triu(matr, k) возвращает верхнюю треугольную матрицу, у которой все элементы над $k$-й диагональю равны 0.

Нумерация диагоналей такая: главная диагональ номер 0, диагонали, расположенные выше главной имеют положительные номера, ниже - отрицательные.



Воспользуемся обеими этими функциями для построения матрицы $L$ с ненулевыми элементами ниже диагонали и диагональной матрицы $D$.

In [ ]:
L = scipy.linalg.tril(A1, -1)
D = scipy.linalg.triu(scipy.linalg.tril(A1, 0), 0)
print(f'A1 = \n{A1}\nL = \n{L}\nD = \n{D}')

A1 = 
[[0.5  0.1  0.   0.   0.  ]
 [0.15 0.5  0.1  0.   0.  ]
 [0.   0.15 0.6  0.1  0.  ]
 [0.   0.   0.15 0.75 0.1 ]
 [0.   0.   0.   0.15 0.55]]
L = 
[[0.   0.   0.   0.   0.  ]
 [0.15 0.   0.   0.   0.  ]
 [0.   0.15 0.   0.   0.  ]
 [0.   0.   0.15 0.   0.  ]
 [0.   0.   0.   0.15 0.  ]]
D = 
[[0.5  0.   0.   0.   0.  ]
 [0.   0.5  0.   0.   0.  ]
 [0.   0.   0.6  0.   0.  ]
 [0.   0.   0.   0.75 0.  ]
 [0.   0.   0.   0.   0.55]]


Вычислим собственные значения матрицы $A_1$ и найдем оптимальное значение $\tau$.

In [ ]:
eigvalsA1 = scipy.linalg.eigvals(A1)
tau = (2 / (min(eigvalsA1) + max(eigvalsA1)))
print(f'tau = {tau} = {tau.real}')

tau = (1.6609704854120044+0j) = 1.6609704854120044


В роли матрицы $P$ метода простой итерации в методе Якоби выступает матрица $(E - D^{-1}A)$, вычислим ее норму для проверки условия сходимости итерационного процесса.

In [ ]:
P2 = (np.eye(A1.shape[0]) - np.linalg.inv(D) @ A1)
np.linalg.norm(P2, ord=np.inf)

0.5

Норма меньше 1, итерационный процесс сходится.
## Пример 3
Решим СЛАУ Примера 1 методом Якоби.


In [23]:
def iteration_Jacobi(A, b, x0, eps=0.001):
    D = scipy.linalg.triu(scipy.linalg.tril(A, 0), 0)
    Dinv = np.linalg.inv(D)
    hatb = Dinv @ b
    P = Dinv @ (D - A)
    P = (np.eye(A1.shape[0]) - Dinv @ A)
    eps1 = eps*(1 - np.linalg.norm(P, ord=np.inf))
    x_prev = x0
    x_next = x0 + np.ones(x0.shape[0])
    while np.linalg.norm(x_prev - x_next, ord=np.inf) >= eps1:
        x_prev = x_next  
        x_next = P @ x_prev + hatb
    return  x_next
x2_k = iteration_Jacobi(A1, b1, b1, eps=0.01)
print(f"""Решение x = {np.round(x2_k, 3).T},
AX - b = {np.round(A1 @ x2_k - b1, 3).T}
|xk - x*| = {np.linalg.norm(x2_k - x1_exact, ord=np.inf).round(3)}""")

Решение x = [2.    3.999 2.    1.999 0.   ],
AX - b = [-0.    -0.    -0.    -0.001 -0.   ]
|xk - x*| = 0.001


## Пример 4
Проверим выполнение условия симметричности и положительной определенности матрицы $A_1$ и единичной матрицы 5 порядка.

Если все собственные значения матрицы положительны, то она положительно определена.

In [24]:
for matr in (A1, np.eye(5)):
    spectr = scipy.linalg.eigvals(matr)
    print(f"""matrix
{matr}
spectr = {np.round(spectr, 2)}
Все с. з. > 0? {np.all(spectr > 0)}
Матрица симметрична? {np.allclose(matr, matr.T)}""")

matrix
[[0.5  0.1  0.   0.   0.  ]
 [0.15 0.5  0.1  0.   0.  ]
 [0.   0.15 0.6  0.1  0.  ]
 [0.   0.   0.15 0.75 0.1 ]
 [0.   0.   0.   0.15 0.55]]
spectr = [0.87+0.j 0.34+0.j 0.68+0.j 0.55+0.j 0.47+0.j]
Все с. з. > 0? True
Матрица симметрична? False
matrix
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
spectr = [1.+0.j 1.+0.j 1.+0.j 1.+0.j 1.+0.j]
Все с. з. > 0? True
Матрица симметрична? True


## Также понадобится:

находить оптимальное значение $\tau$ можно, например, так: 

In [ ]:
tau = 2 / (sum([item(scipy.linalg.eigh(A1, eigvals_only=True)) for item in (min, max)]))
tau

1.6630712616119498

Пусть описана функция с именем f_name. Как вывести на экран ее имя?

In [ ]:
def f_name():
  return True
print(f'Имя функции {f_name.__name__}')  

Имя функции f_name
